In [1]:
import pandas as pd
import os
import sys
sys.path.append("../")


import glob
import pandas as pd

In [2]:
with open('D:\M1/fashion/lda_model/data/fashion_clip.txt', mode="r", encoding="utf-8") as f:
    data = f.read().splitlines()
item_info = {}
for i in data:
    i = i.strip()
    item_id, category, color = i.split(' ')
    item_info[int(item_id)] = f'{category} {color}'


In [11]:
from create_data.create_compatibility import is_bottoms, is_shoes, is_tops


count = {}
with open('D:\M1/fashion/lda_model/data/filepath/train.txt', 'r', encoding="utf-8") as f:
    filepaths = f.read().splitlines()

file_count = len(filepaths)
output_file = "D:\M1/fashion/lda_model/data/fashion_clip/train_with_tbs.txt"
c = 0
skip_ids = set()
with open(output_file, mode="w", encoding="utf-8") as f:
    for fp in filepaths:
        c += 1
        try:
            json_dict = pd.read_json(fp, encoding='shift-jis')
        except Exception as e:
            continue
        
        tops = []
        bottoms = []
        shoes = []
        for item in json_dict["items"]:
            if ("itemId" not in item or item["itemId"] == None):
                continue
            item_id = item["itemId"]
            if item_id not in item_info:
                skip_ids.add(item_id)
                continue
            if "category x color" not in item:
                continue
            category = item["category x color"].split(' ')[0]
            if is_tops(category):
                tops.append(item_info[item_id])
            elif is_bottoms(category):
                bottoms.append(item_info[item_id])
            elif is_shoes(category):
                shoes.append(item_info[item_id])
        for t in tops:
            for b in bottoms:
                for s in shoes:
                    words = [t, b, s]
                    document = " ".join(words) + "\n"
                    f.write(document)
        if (c % 100 == 0):
            print(f"{c * 100 / file_count}%終了 skip_cnt: {len(skip_ids)}")


0.04627444447529407%終了 skip_cnt: 142
0.09254888895058815%終了 skip_cnt: 294
0.1388233334258822%終了 skip_cnt: 445
0.1850977779011763%終了 skip_cnt: 577
0.23137222237647037%終了 skip_cnt: 625
0.32392111132705853%終了 skip_cnt: 730
0.3701955558023526%終了 skip_cnt: 762
0.46274444475294074%終了 skip_cnt: 802
0.5090188892282348%終了 skip_cnt: 852
0.5552933337035288%終了 skip_cnt: 897
0.601567778178823%終了 skip_cnt: 947
0.6478422226541171%終了 skip_cnt: 1065
0.6941166671294111%終了 skip_cnt: 1166
0.7403911116047052%終了 skip_cnt: 1200
0.7866655560799992%終了 skip_cnt: 1271
0.8329400005552934%終了 skip_cnt: 1356
0.8792144450305874%終了 skip_cnt: 1406
0.9254888895058815%終了 skip_cnt: 1493
0.9717633339811755%終了 skip_cnt: 1554
1.0180377784564696%終了 skip_cnt: 1666
1.0643122229317636%終了 skip_cnt: 1704
1.1105866674070577%終了 skip_cnt: 1729
1.1568611118823517%終了 skip_cnt: 1767
1.203135556357646%終了 skip_cnt: 1814
1.24941000083294%終了 skip_cnt: 1853
1.3419588897835282%終了 skip_cnt: 1942
1.3882333342588222%終了 skip_cnt: 1987
1.434507778

In [13]:
with open('D:\M1/fashion/lda_model/data/skip_ids.txt', mode="w", encoding="utf-8") as f:
    for i in list(skip_ids):
        f.write(f'{i} \n')